In [1]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
from google.colab import drive
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
df = pd.read_csv('chunked_news.csv')
df

,Date,Reversal_Label,combined_headlines
0,2023-11-15,No Reversal,WIPO Publishes Patent of DIRECTA PLUS S.P.A. w...
1,2023-11-15,No Reversal,Invesco S&P 500 Downside Hedged (PHDG:$32.61) ...
2,2023-11-15,No Reversal,Invesco S&P SmallCap Momentum (XSMO:$49.95) fa...
3,2023-11-15,No Reversal,Lyxor S&P 500 UCITS - Daily Hedged D-EUR (SP5H...
4,2023-11-15,No Reversal,iShares Core S&P BSE SENSEX India (2836:HKD35....
...,...,...,...
2975,2025-02-13,No Reversal,Top Performers Past Week: Palantir Technologie...
2976,2025-02-13,No Reversal,S&P 500 Consumer Staples (Sector): The Top Fiv...
2977,2025-02-13,No Reversal,General Mills offers 46th lowest Price Earning...
2978,2025-02-13,No Reversal,SPDR S&P MidCap 400 (MDY:$579.11) in 2nd conse...


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Preprocess the data
label_map = {'No Reversal': 0, 'Positive': 1, 'Negative': 2}
df['label'] = df['Reversal_Label'].map(label_map)

# Split the data into training, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['combined_headlines'], df['label'], test_size=0.3)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

# Tokenize the text data
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512)

# Convert the data into torch tensors
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label_ids'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels.tolist())
val_dataset = NewsDataset(val_encodings, val_labels.tolist())
test_dataset = NewsDataset(test_encodings, test_labels.tolist())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

In [ ]:
drive.mount('/content/gdrive')

torch.backends.cudnn.benchmark = True

# Load the FinBERT model
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1,2]),
    y=train_labels.to_numpy()  # Assuming labels are stored in train_dataset
)

# Convert to tensor
class_weights = torch.tensor(class_weights, dtype=torch.float32).to("cuda")

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)  # Apply class weights
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Define the training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/finbert_checkpoints',
    num_train_epochs=10,  # You can reduce this to 1 or 2 to save time
    per_device_train_batch_size=32,  # Reduce batch size to save memory
    per_device_eval_batch_size=32,  # Reduce batch size to save memory
    warmup_steps=500,
    max_steps=1000,  # Increase max steps to save time
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    lr_scheduler_type="cosine",  # Use cosine learning rate scheduler
    learning_rate=1e-6,
    fp16 = True,
    load_best_model_at_end=True
)

# Create the Trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Add the callback here
)

trainer.train()
trainer.evaluate()



print("Training complete.")

# Evaluate the model on the test set
print("Evaluating on the test set...")
test_results = trainer.evaluate(test_dataset)
print(f"Test results: {test_results}")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Epoch,Training Loss,Validation Loss
1,5.490100,4.575447
2,4.674900,3.545469
3,2.635500,2.524021
4,1.698400,1.637577
5,1.301700,1.280156
6,1.229700,1.162925
7,1.156300,1.139534
8,1.172600,1.162664
9,1.171500,1.131950
10,1.146800,1.125498


Training complete.
Evaluating on the test set...
Test results: {'eval_loss': 1.1265432834625244, 'eval_runtime': 3.0275, 'eval_samples_per_second': 147.646, 'eval_steps_per_second': 4.624, 'epoch': 15.151515151515152}


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)  # Get predicted class
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("Evaluating on the test set...")
test_results = trainer.evaluate(test_dataset)

# Print evaluation metrics
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Precision: {test_results['eval_precision']:.4f}")
print(f"Recall: {test_results['eval_recall']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")


Evaluating on the test set...


Accuracy: 0.3468
Precision: 0.7244
Recall: 0.3468
F1 Score: 0.4259
